# Day 14

## Part I

第一部分直接实现，此处使用了Numpy数组来加快速度，然后使用两个mask，可以直接使用二进制与和或运算直接获得结果：

In [1]:
import numpy as np

class DockingProgram(object):
    def __init__(self, mem_size: int = 64 * 1024):
        self.memory = np.zeros(mem_size, dtype='i8')
        self.mask_0 = 1 << 36 - 1
        self.mask_1 = 0
    def set_mask(self, mask: str):
        mask_0 = mask.replace('X', '1')
        mask_1 = mask.replace('X', '0')
        self.mask_0 = int(mask_0, 2)
        self.mask_1 = int(mask_1, 2)
    def set_memory(self, index: str, value: str):
        value = int(value) & self.mask_0 | self.mask_1
        self.memory[int(index)] = value
    def memory_sum(self) -> int:
        return self.memory.sum()
    def run(self, command: str):
        parts = command.rstrip().split(' = ')
        if parts[0] == 'mask':
            self.set_mask(parts[1])
        elif parts[0][:3] == 'mem':
            self.set_memory(parts[0][4:-1], parts[1])

读取输入文件函数和第一部分实现的逻辑：

In [2]:
from typing import List

def read_input(input_file: str) -> List[str]:
    with open(input_file) as fn:
        return fn.readlines()

def part1_solution(commands: List[str]) -> int:
    program = DockingProgram()
    for command in commands:
        program.run(command)
    return program.memory_sum()

单元测试：

In [3]:
testcase = read_input('testcase1.txt')
assert(part1_solution(testcase) == 165)

然后得到第一部分结果：

In [4]:
commands = read_input('input.txt')
part1_solution(commands)

3059488894985

## Part II

第二部分稍微复杂一点，首先定义一个BitMask类来实现对内存地址的mask操作，此处使用了Numpy数组本身的各种选择操作：

In [5]:
class BitMask(object):
    def __init__(self, mask: str):
        self.mask = np.array(list(mask))
    def do_mask(self, value: str) -> List[int]:
        value = np.array(list(np.binary_repr(int(value)).zfill(36)))
        for index, bit in enumerate(self.mask):
            if bit == '1' or bit == 'X':
                value[index] = bit
        xcount = (value=='X').sum()
        result = []
        for i in range(1<<xcount):
            xs = np.binary_repr(i).zfill(xcount)
            v = value.copy()
            v[v=='X'] = np.array(list(xs))
            result.append(int(''.join(v.tolist()), 2))
        return result

然后先对BitMask进行单元测试：

In [6]:
bm = BitMask('000000000000000000000000000000X1001X')
assert(bm.do_mask('42') == [26, 27, 58, 59])

bm = BitMask('00000000000000000000000000000000X0XX')
assert(bm.do_mask('26') == [16, 17, 18, 19, 24, 25, 26, 27])

下面需要重新定义DockingProgram类，创建一个新的DockingProgram2类，这里的内存内容无法再使用Numpy数组来加速，因为地址可能高达36bit长度，所以，为了节省内存，这里采用了字典来作为memory的类型，同时牺牲了一定性能：

In [7]:
class DockingProgram2(object):
    def __init__(self, mem_size: int = 64 * 1024, bit_size: int=36):
        self.memory = {}
        self.mask = BitMask('0' * bit_size)
    def set_mask(self, mask: str):
        self.mask = BitMask(mask)
    def set_memory(self, index: str, value: str):
        indices = self.mask.do_mask(index)
        for index in indices:
            self.memory[index] = int(value)
    def memory_sum(self) -> int:
        return sum(self.memory.values())
    def run(self, command: str):
        parts = command.rstrip().split(' = ')
        if parts[0] == 'mask':
            self.set_mask(parts[1])
        elif parts[0][:3] == 'mem':
            self.set_memory(parts[0][4:-1], parts[1])

第二部分的实现逻辑：

In [8]:
def part2_solution(commands: List[str]) -> int:
    program = DockingProgram2()
    for command in commands:
        program.run(command)
    return program.memory_sum()

第二部分单元测试：

In [9]:
testcase2 = read_input('testcase2.txt')
assert(part2_solution(testcase2) == 208)

最后得到第二部分的结果：

In [10]:
part2_solution(commands)

2900994392308